<a href="https://colab.research.google.com/github/spearminty/CShell/blob/master/camogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

StyleGAN2-ADA only works with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [3]:
#core configurations 
dataset_name = 'camogen'
resize_value = 512
#used to identify a distinct lineage of training checkpoints
network_name = 'network1'
#use this to skip training epochs from script
skip_training = False


In [1]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Fri Jun 24 03:40:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

In [2]:
from google.colab import drive
from pathlib import Path
import os
content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Mounted at /content/drive


run this cell to install StyleGAN2-ADA on your Drive. If you’ve already installed the repository, it will skip the installation process and only check for updates. If you haven’t installed it, it will install all the necessary files. Beside, in, out, datasets and training folders are generated for data storage. Everything will be available on your Google Drive in the folder StyleGAN2-ADA even after closing this Notebook.

In [4]:
stylegan2_repo_url  = 'https://github.com/dvschultz/stylegan2-ada' # or https://github.com/NVlabs/stylegan2-ada

project_path        = drive_path / 'MyDrive' / 'train' / 'project' / dataset_name
image_path          = drive_path / 'MyDrive' / 'train' / 'images' / 'camo1'
stylegan2_repo_path = project_path / 'stylegan2-ada'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Download StyleGAN2-ada
!git config --global user.name "spearminty"
!git config --global user.email "thx1134@gmail.com"
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}

/content/drive/MyDrive/train/project/camogen


Train a custom model

resize the images

In [5]:
from tqdm import tqdm
from PIL import Image
training_resize_path = image_path / str(resize_value)

print(f"Looking for folder: {training_resize_path}")

if not os.path.isdir(training_resize_path):
    os.mkdir(training_resize_path)
    print("Resizing training images...")

# this resizes to square. consider cropping square first (if not square) then resize
    training_data = []
    iter = 0
    for filename in tqdm(os.listdir(image_path)):
        if (image_path / filename).is_dir():
          continue
        print(filename)
        path = os.path.join(image_path,filename)
        image = Image.open(path).resize((resize_value,
            resize_value),Image.ANTIALIAS)
        save_path = training_resize_path / filename
        image.save(save_path)

Looking for folder: /content/drive/MyDrive/train/images/camo1/512


process dataset and convert to tfr format

In [6]:
local_images_path = image_path / str(resize_value)
local_dataset_path = project_path / 'tfr'

if (local_dataset_path).is_dir():
    print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
    print('Delete current dataset folder ({}) to regenerate tfrecords.'.format(local_dataset_path))
else:
    %mkdir "{local_dataset_path}"
    !python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images \
        "{local_dataset_path}" "{local_images_path}"

❗ Dataset already created ❗
Delete current dataset folder (/content/drive/MyDrive/train/project/camogen/tfr) to regenerate tfrecords.


here are numerous arguments to tune the training of your model. To obtain nice results, you will certainly have to experiment. Here are the most popular parameters:

mirror: Should the images be mirrored vertically?

mirrory: Should the images be mirrored horizontally?

snap: How often should the model generate image samples and a network pickle (.pkl file)?

resume: Network pickle to resume training from?

To see all the options, run the following help cell.

Please note that Google Colab Pro gives access to V100 GPUs, which drastically decreases (~3x) processing time over P100 GPUs.

In [ ]:
#!python "{stylegan2_repo_path / 'train.py'}" --help

In [8]:
import glob

def locate_latest_network(training_path):
    latest_file = None
    print(f"Scanning {training_path}..")
    if training_path.is_dir():
      list_of_files = glob.glob(f'{training_path}/**/*.pkl')
      if len(list_of_files)>0:
          latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    return latest_file

In [ ]:
if skip_training:
  exit()
#tensor 1.x requires numpy 1.16 to function. 
#we just live with the error messages here and do not restart
!pip uninstall -y numpy
!pip install numpy==1.16.4

import numpy
print(numpy.__version__)

training_path = project_path / 'training' / dataset_name / network_name
if not training_path.is_dir():
    %mkdir "{training_path}"

#how often should the model generate samples and a .pkl file
snapshot_count = 10
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = 'bgc'
snap = 10
resume_arg = ''

#locate latest resume from
latest_file = locate_latest_network(training_path)
if latest_file != None:
    resume = training_path / latest_file
    print (f'located a resume from network for {network_name}: {latest_file} ')
    resume_arg = f' --resume={resume}' 
    print(f'{resume_arg}')
print(f'{local_dataset_path}')

!python "{stylegan2_repo_path / 'train.py'}" --outdir="{training_path}" \
    --data="{local_dataset_path}" {resume_arg} \
    --snap={snapshot_count} --augpipe={augs} \
    --snap={snap} \
    --mirror={mirrored} --mirrory={mirroredY} \
    --metrics={metric_list} #--dry-run

Found existing installation: numpy 1.21.6
Uninstalling numpy-1.21.6:
  Successfully uninstalled numpy-1.21.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.3 MB 724 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
xarray 0.20.2 requires numpy>=1.18, but you have numpy 1.16.4 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.16.4 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.16.4 which is incompatible.
scikit-image 0.18.3 requires numpy>=1.16.5, but you have numpy 1.16.4 which is incompatible.
pywavelets 1.3.0 requires numpy>=1.17.3, but you have num

1.21.6
Scanning /content/drive/MyDrive/train/project/camogen/training/camogen/network1..
/content/drive/MyDrive/train/project/camogen/training/camogen/network1/00001-tfr-mirror-auto1-bgc-resumecustom/network-snapshot-000040.pkl
located a resume from network for network1: /content/drive/MyDrive/train/project/camogen/training/camogen/network1/00001-tfr-mirror-auto1-bgc-resumecustom/network-snapshot-000040.pkl 
 --resume=/content/drive/MyDrive/train/project/camogen/training/camogen/network1/00001-tfr-mirror-auto1-bgc-resumecustom/network-snapshot-000040.pkl
/content/drive/MyDrive/train/project/camogen/tfr

Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 16384,
    "fmap_max": 512,
    "mapping_layers": 2,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 16384,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_

You can finally generate images using a pre-trained network once everything is set-up. 

In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

In [ ]:
from numpy import random
import os
seed_init = random.randint(10000)
nbr_images = 6
generation_from=None

#uncomment below to enable this code block to generate images directly by training pickle file path
#generation_from = '/content/drive/MyDrive/train/project/camogen/out/012.pkl'

stylegan2_repo_url  = 'https://github.com/dvschultz/stylegan2-ada' # or https://github.com/NVlabs/stylegan2-ada
stylegan2_repo_path = '/content/drive/MyDrive/train/project/camogen/stylegan2-ada'

from google.colab import drive
drive.mount('/content/drive')
%tensorflow_version 1.x
!nvidia-smi
# Download StyleGAN2-ada
!git config --global user.name "spearminty"
!git config --global user.email "thx1134@gmail.com"
if os.path.exists(stylegan2_repo_path):
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}
%pip install opensimplex

if generation_from == None:
  latest_file = locate_latest_network(training_path)
  if latest_file != None:
      generation_from = training_path / latest_file
  if generation_from == None:
    print("Could not locate valid network checkpoint!")
    exit()

!python "{os.path.join(stylegan2_repo_path, 'generate.py')}" generate-images \
      --outdir="{'/content/drive/MyDrive/train/project/camogen/out'}" --trunc=0.7 \
      --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
      --network={generation_from}